In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import base64
import time
import random
from bs4 import BeautifulSoup


def get_access_token():
    """获取百度OCR的AccessToken"""

    # 百度AI配置（请替换为你的API信息）
    API_KEY = "pnBkPfBD2REvCbjhgOmHd9wo"
    SECRET_KEY = "pQ2HEtOz5Vqlt5VylH158aN82QO0Hh57"

    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    response = requests.post(url, params=params)
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"获取AccessToken失败: {response.text}")



def recognize_captcha(image_path=None, image_data=None):
    """
    使用百度OCR识别验证码
    参数:
        image_path: 图片文件路径
        image_data: 图片二进制数据
    返回:
        识别结果文本
    """
    # 获取AccessToken
    access_token = get_access_token()
    url = f"https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic?access_token={access_token}"
    
    # 准备图片数据
    if image_path:
        with open(image_path, "rb") as f:
            image = base64.b64encode(f.read()).decode("utf-8")
    elif image_data:
        image = base64.b64encode(image_data).decode("utf-8")
    else:
        raise ValueError("必须提供image_path或image_data")
    
    # 构建请求
    payload = {
        "image": image,
        "detect_direction": "false",
        "detect_language": "false",
        "paragraph": "false",
        "probability": "false"
    }
    
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    }
    
    # 发送请求
    response = requests.post(url, headers=headers, data=payload)
    
    # 处理响应
    if response.status_code == 200:
        result = response.json()
        if "words_result" in result and len(result["words_result"]) > 0:
            return result["words_result"][0]["words"].strip()
        else:
            print(f"OCR识别失败: {result}")
            return None
    else:
        raise Exception(f"OCR API调用失败: {response.text}")


def extract_available_appointments_doctor(html_str, doctor_name="袁玲玲"):
    # 解析HTML
    soup = BeautifulSoup(html_str, 'lxml')
    table = soup.find('table')
    
    # 存储可预约的条目
    available_entries = []
    
    # 获取表头（th标签）作为字段名
    headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]
    
    # 查找"医生姓名"在表头中的索引位置
    try:
        doctor_index = headers.index('医生姓名')
    except ValueError:
        print("警告: 表格中未找到'医生姓名'列")
        return available_entries
    
    # 遍历每一行（跳过表头）
    for row in table.find_all('tr')[1:]:
        cells = row.find_all('td')
        if not cells:  # 跳过空行
            continue
            
        # 提取每列数据
        data = {headers[i]: cells[i].get_text(strip=True) for i in range(len(headers))}
        
        # 检查医生姓名是否匹配
        if data.get('医生姓名') != doctor_name:
            continue
        
        # 检查操作列是否有符合条件的链接
        operation_cell = cells[-1]  # 操作列是最后一列
        link = operation_cell.find('a', class_='dailySchedulingState_a_4')
        
        if link and '/Interactions/SchedulingAppointments/OPCreate' in link.get('href', ''):
            # 提取预约链接
            appointment_url = "https://www.dzmyy.com.cn" + link.get('href')
            data['预约链接'] = appointment_url
            
            # 添加到结果列表
            available_entries.append(data)
    
    return available_entries


logOnUrl = "https://www.dzmyy.com.cn/Account/LogOn"
skinDepartUrl = "https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPDoctorIndex?OPdepartmentId=270"    


options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(options=options)
driver.get(logOnUrl)

try:
    # 等待用户名、密码元素加载
    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'UserName'))
    )
    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'Password_pwd'))
    )
    
    # 填写用户名和密码
    username_input.send_keys('18811787891')
    password_input.send_keys('18811787891xpc')
    # time.sleep(random.uniform(1, 2))  # 模拟人类输入延迟
    
    # 验证码
    captcha_img = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'valiCodeLogOnCode')) 
    )
    captcha_img.click()
    time.sleep(random.uniform(1, 2))  # 模拟人类输入延迟
    captcha_img.screenshot("captcha_local.png") 
    
    # 识别验证码
    captcha_text = recognize_captcha(image_path='captcha_local.png')
    if not captcha_text:
        print("验证码识别失败，退出登录流程")
        driver.quit()  # 关闭浏览器
    
    print(f"识别的验证码: {captcha_text}")

    # 验证码输入框
    LogOnCode_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'LogOnCode'))
    )
    LogOnCode_input.clear()
    LogOnCode_input.send_keys(captcha_text)
    time.sleep(random.uniform(1, 2))
    
    # 点击登录
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#con_tableb_1 > input'))
    )
    login_button.click()
    
    # 切换到皮肤科室预约url
    driver.get(skinDepartUrl)
    try:
        # 等待页面加载完成，等待表格元素出现
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # 获取页面源代码
        page_source = driver.page_source
        
        # 调用函数获取可预约信息
        available_appointments = extract_available_appointments_doctor(page_source, doctor_name="袁玲玲")
        
        # 打印结果
        print(f"找到 {len(available_appointments)} 个可预约时段：")
        for appointment in available_appointments:
            print(f"日期: {appointment.get('日期', 'N/A')}")
            print(f"时段: {appointment.get('时段', 'N/A')}")
            print(f"预约链接: {appointment.get('预约链接', 'N/A')}")
            
            # 切换到预约页面
            driver.get(appointment.get('预约链接', 'N/A'))
            time.sleep(random.uniform(1, 2))
            # current_url is the new url after redirect
            current_url = driver.current_url
            print(f"当前页面URL: {current_url}")
            
            # 等待预约按钮出现
            appointment_buttons = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span.SchedulingState_icon4.outpatient[title='预约']"))
            )
            
            if appointment_buttons:
                print(f"找到 {len(appointment_buttons)} 个可预约按钮")
                # 选择第一个可用的预约按钮
                appointment_buttons[0].click()
                print("已点击第一个预约按钮")
                # 现在进入一个新的页面，该页面有一个“确认预约”的按钮，请帮我点击这个按钮
                confirm_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='submit'][value='确认预约'].achedulingAppointment_btn"))
                )
                confirm_button.click()
                print("已确认预约")
                
            print("-" * 50)
            
    except Exception as e:
        print(f"获取预约信息时发生错误: {e}")
except Exception as e:
    print(f"登录过程发生错误: {e}")
finally:
    input("按Enter键关闭浏览器...")
    driver.quit() 


Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)
The chromedriver version (136.0.7103.113) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (137.0.7151.41); currently, chromedriver  is recommended for chrome 137.*, so it is advised to delete the driver in PATH and retry


识别的验证码: 119853
找到 7 个可预约时段：
日期: N/A
时段: 上午
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2116192
当前页面URL: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/NumList?schId=2116192&depId=270&docId=3241&parameters={%22node_id%22:%22%22,%22res_id%22:%22%22}
找到 2 个可预约按钮
已点击第一个预约按钮
已确认预约
--------------------------------------------------
日期: N/A
时段: 上午
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2118028
当前页面URL: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPSuccess?schedulingAppointmentId=278077789295608832
获取预约信息时发生错误: Message: 

